In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score



%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test = pd.read_csv("../input/loan-default-prediction/test_v2.csv.zip", low_memory=False)
train = pd.read_csv("../input/loan-default-prediction/train_v2.csv.zip", low_memory=False)


In [ ]:
numerical = train.select_dtypes(include=[np.number])
numerical_cols = numerical.columns.values

for col in numerical_cols:
    miss = train[col].isnull()
    num_miss = np.sum(miss)
    
    if num_miss > 0:
        train['{}_ismissing'.format(col)] = miss
        mean_val = train[col].mean()
        train[col] = train[col].fillna(mean_val)

In [ ]:
test_numerical = test.select_dtypes(include=[np.number])
test_numerical_cols = test_numerical.columns.values

for col in test_numerical_cols:
    miss = test[col].isnull()
    num_miss = np.sum(miss)
    
    if num_miss > 0:
        test['{}_ismissing'.format(col)] = miss
        mean_val = test[col].mean()
        test[col] = test[col].fillna(mean_val)

In [ ]:
train.drop(train.iloc[:, 771:1284], inplace = True, axis = 1) 
test.drop(test.iloc[:, 771:1284], inplace = True, axis = 1) 

In [ ]:
correl_matrix = train.corr().abs()
upper = correl_matrix.where(np.triu(np.ones(correl_matrix.shape), k=1).astype(np.bool))
drop = [column for column in upper.columns if any(upper[column] > 0.95)]
train.drop(drop, axis=1, inplace=True)
#also dropping these columns in the test df
test.drop(drop, axis=1, inplace=True)

In [ ]:
train['loss_factor'] = train['loss']

loss_factor = train['loss']

for i in loss_factor:
    if i != 0:
        loss_factor = loss_factor.replace(i, 1)
        
train['loss_factor'] = loss_factor

In [ ]:
train = pd.concat([train.select_dtypes(include=[np.float64]), train['loss_factor']], axis=1)

test = test.select_dtypes(include=[np.float64])

In [ ]:
test_flt_train = test
test_flt_train = test_flt_train.drop('f5', axis = 1)

In [ ]:
train = train.dropna(how='any')

In [ ]:
x= train.drop(['loss_factor'], axis=1)
y= train['loss_factor']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=15)
x_train.shape, x_test.shape

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
classifier=model.fit(x_train,y_train)

In [ ]:
prediction = model.predict(x_test)

In [ ]:
acc = accuracy_score(y_test, prediction)

In [ ]:
print(acc*100)

In [ ]:
def submission_write_file(predicted_labels, out_file, train_num=105471,
                    target='loss', index_label="id"):
    #turning predictions into a data frame and saving them as a csv file
    predicted_data = pd.DataFrame(predicted_labels,
                                index = np.arange(train_num + 1,
                                                  train_num + 1 +
                                                  predicted_labels.shape[0]),
                                columns=[target])
    predicted_data.to_csv(out_file, index_label=index_label)

In [ ]:
classifier.predict(test_flt_train)

In [ ]:
submission_write_file(classifier.predict(test_flt_train), out_file="decision_Tree_loan_prediction.csv")